In [1]:
import os
import re
import skbio
import numpy as np
import pandas as pd
import xlrd
import seaborn as sns
from scipy import stats
import matplotlib as mpl
from scipy.spatial import distance
from matplotlib import pyplot as plt
from meta.scripts.Utilities import Utilities

In [2]:
def process_sample_number(s: str):
    d = dict()
    d["sample_number"], group_code = [i.strip() for i in s.split("/")]
    if group_code[0] == "В":
        d["age"] = "adult"
    if group_code[1] == "О":
        d["group"] = "obesity"
    elif group_code[1] == "З":
        d["group"] = "normal"
    return pd.Series(d)


def fix_exponentials(s: str):
    s = str(s).strip().lower().replace(",", ".")
    try:
        return float(s)
    except ValueError:
        base, power = [float(i) for i in re.split("[^0-9.]+", s)]
        f = base * (10 ** power)
        if "-" in s:
            f = base / (10 ** power)
        return f


def get_statistical_test_series(two_dim_array: list, func=stats.mannwhitneyu, alpha: float = 0.05):
    # Use with the 'stats' package only!
    p_value = 1
    if sum(map(sum, two_dim_array)) > 0:
        p_value = func(*two_dim_array).__getattribute__("pvalue")
    d = dict(p_value=p_value, is_significant_for_single_comparison=p_value < alpha)
    return pd.Series(d)


def safe_capitalize(s: str):
    if len(s) < 2:
        return s.upper()
    return "".join([s[0].upper(), s[1:]])


In [3]:
PROJECT_DIR = "/data1/bio/projects/ashestopalov/nutrition/obesity_elisa"
COMMON_CLINICAL_DATA_COL_NAMES = {
    "имт": "Body Mass Index", "от": "Waist Circumference", "глюкоза": "Glucose", 
    "лпвп": "High-Density Lipoproteins", "тг": "Thyroglobulin", 
    "ох": "Total Blood Cholesterol", "лпнп": "Low-Density Lipoproteins", 
    "сад": "Systolic Blood Pressure", "дад": "Diastolic Blood Pressure"
}
CORRELATION_TITLE_TEMPLATE = "Correlation between {} for {}"
PAIRPLOT_TITLE_TEMPLATE = "Pair plot for {} for {}"

raw_data_dir = os.path.join(PROJECT_DIR, "raw")
correlation_dir = os.path.join(PROJECT_DIR, "correlation")
otu_dir = os.path.join(PROJECT_DIR, "otu")

whole_raw_df = pd.DataFrame()
for raw_table_file in Utilities.scan_whole_dir(raw_data_dir):
    if "obschaya" in raw_table_file and "kontrol'" not in raw_table_file:
        continue
    raw_table_df = pd.read_excel(raw_table_file, encoding="utf-8").dropna(axis=0, how="all").dropna(
        axis=1, how="all")
    raw_table_df = raw_table_df.rename(columns={"охс": "ох"}).rename(
        columns=COMMON_CLINICAL_DATA_COL_NAMES)
    raw_table_df = pd.concat([raw_table_df, raw_table_df["№ образца"].apply(process_sample_number)],
                             axis=1, sort=False)
    if "MZO" in raw_table_file:
        raw_table_df["subgroup"] = "metabolically-healthy_obesity"
    elif "MNZ" in raw_table_file:
        raw_table_df["subgroup"] = "metabolically-pathological_obesity"
    elif "wzroslye s ozhireniem'" in raw_table_file:
        raw_table_df["group"] = "obesity"
    elif "kontrol'" in raw_table_file:
        raw_table_df["group"] = "normal"
    raw_table_df["sex"] = raw_table_df["пол"].apply(
        lambda x: ["male", "female"][x.strip().lower() == "ж"])
    raw_table_df["patient_id"] = raw_table_df["sample_number"].str.lower().replace(
        '["b]+', "", regex=True)
    raw_table_df = raw_table_df.set_index("sample_number")
    whole_raw_df = pd.concat([whole_raw_df, raw_table_df], axis=0, sort=False).rename_axis(
        index="sample_number", columns="values")
    

raw_clinical_data_col_names = sorted(COMMON_CLINICAL_DATA_COL_NAMES.values())
raw_multiplex_data_col_names = [i for i in whole_raw_df.columns if i[0].isupper()]
raw_value_col_names = raw_multiplex_data_col_names + raw_clinical_data_col_names

whole_raw_df.loc[:, raw_value_col_names] = whole_raw_df.loc[:, raw_value_col_names].replace(
    [np.inf,-np.inf], np.nan).applymap(fix_exponentials)
whole_raw_df.drop(columns=[i for i in whole_raw_df.columns if len(re.findall(
    "^[a-z]", i.lower())) == 0], inplace=True)

group_names = sorted(set(whole_raw_df["group"].values))
obesity_subgroup_names = sorted(set(whole_raw_df["subgroup"].dropna().values))

In [4]:
def drop_static_entries(df: pd.DataFrame(), axis: int = 1):
    d = df.std(axis=int(not bool(axis))).to_dict()
    static_axis = [k for k in d.keys() if d[k] == 0]
    return df.drop(static_axis, axis=axis)

def prepare_df(data: pd.DataFrame, value_col_names: list):
    common_col_names = data.columns
    if len(value_col_names) > 0:
        common_col_names = np.intersect1d(data.columns, value_col_names)
    numeric_columns = data.select_dtypes(include=np.number).columns
    out = drop_static_entries(
        data.loc[:, np.intersect1d(numeric_columns, common_col_names)].fillna(0))
    return out

def export_plot(basename: str):
    plt.tight_layout()
    os.makedirs(os.path.dirname(basename), exist_ok=True)
    plt.savefig("{}.pdf".format(basename), dpi=600)
    mpl.rcParams.update(mpl.rcParamsDefault)
    plt.clf()
    plt.close()

def build_clustermap(data: pd.DataFrame, title: str, out_dir: str, value_col_names: list = ()):
    export_prefix = os.path.join(out_dir, title)
    _data = prepare_df(data, value_col_names)
    correlation_df = _data.corr(method="spearman")
    Utilities.dump_tsv(correlation_df, "{}.tsv".format(export_prefix))
    plt.rcParams["figure.figsize"] = (10, 10)
    sns.set()
    _ = sns.clustermap(correlation_df, metric="cityblock", cmap="Spectral_r").fig.suptitle(
        title, fontsize=10,y=0.995) 
    export_plot(export_prefix)

def build_pairplot(data: pd.DataFrame, title: str, out_dir: str, value_col_names: list = ()):
    export_prefix = os.path.join(out_dir, title)
    _data = prepare_df(data, value_col_names)
    Utilities.dump_tsv(_data, "{}.tsv".format(export_prefix))
    plt.rcParams["figure.figsize"] = (20, 20)  
    sns.set()
    _ = sns.pairplot(data=_data, kind="reg", vars=_data.columns).fig.suptitle(
        title, fontsize=10, y=0.997)
    export_plot(export_prefix)

In [5]:
# Boilerplate1
# Build clinical - ELISA data correlation
data_prefix = "clinical and ELISA data"
for group_name in group_names:
    group_df = whole_raw_df.query("group == '{}'".format(group_name)).rename_axis(
        columns=safe_capitalize(data_prefix))
    build_clustermap(data=group_df, value_col_names=raw_value_col_names, out_dir=correlation_dir, 
                     title=CORRELATION_TITLE_TEMPLATE.format(data_prefix, group_name))
    build_pairplot(data=group_df, value_col_names=raw_value_col_names, out_dir=correlation_dir, 
                   title=PAIRPLOT_TITLE_TEMPLATE.format(data_prefix, group_name))

for obesity_subgroup_name in obesity_subgroup_names:
    obesity_subgroup_df = whole_raw_df.query(
        "subgroup == '{}'".format(obesity_subgroup_name)).rename_axis(
        columns=safe_capitalize(data_prefix))
    build_clustermap(data=obesity_subgroup_df, value_col_names=raw_value_col_names, 
                     out_dir=correlation_dir, 
                     title=CORRELATION_TITLE_TEMPLATE.format(data_prefix, obesity_subgroup_name))
    build_pairplot(data=obesity_subgroup_df, value_col_names=raw_value_col_names, 
                   out_dir=correlation_dir, 
                   title=PAIRPLOT_TITLE_TEMPLATE.format(data_prefix, obesity_subgroup_name))

In [6]:
# Download trees from 'ftp://greengenes.microbio.me/greengenes_release/gg_13_5/gg_13_8_otus.tar.gz'
otu_97_tree = skbio.TreeNode.read(os.path.join(otu_dir, "97_otus.tree"), verify=True)
otu_97_annotation_df = pd.read_csv(os.path.join(otu_dir, "97_otu_taxonomy.txt"), header="infer", 
                                   names=["id", "taxon"], sep="\t")
otu_97_annotation_df.loc[:, "id"] = otu_97_annotation_df["id"].astype(str)
otu_97_annotation_df.set_index("id", inplace=True)
otu_97_annotation_dict = {re.sub(";([ ]+)", ";", i): j for i, j in zip(
    otu_97_annotation_df["taxon"],otu_97_annotation_df.index.values)}

# Assign tree node length even for roots    
otu_97_tree_node_names = [i.name for i in otu_97_tree.postorder()]
for otu_97_node in otu_97_tree.postorder():
    if otu_97_node.length is None:
        otu_97_node.length = 0.0

In [7]:
def fit_df_to_tree(df: pd.DataFrame):
    df = df.rename(columns=otu_97_annotation_dict).fillna(0)
    sub_df = df.drop([i for i in df.columns if i not in otu_97_tree_node_names], axis=1)
    return sub_df

def mp_count_faith_pd(series: pd.Series):
    x = skbio.diversity.alpha.faith_pd(series.values, series.index.values, otu_97_tree)
    return pd.Series({"Faith Diversity": x, "patient_id": series.name})

In [8]:
TAXONOMIC_RANKS = ["Phyla", "Class", "Order", "Order", "Genus", "Species"]

raw_stool_otu_df = pd.read_excel(os.path.join(otu_dir, "Results_Rogachev_stool_final.xlsx"), 
                                 encoding="utf-8", sheet_name=TAXONOMIC_RANKS[-1], 
                                 index_col='#OTU ID"')
raw_blood_otu_df = pd.read_excel(os.path.join(otu_dir, "Results_16s_Rogachev_blood_291219.xlsx"), 
                                 encoding="utf-8", sheet_name=TAXONOMIC_RANKS[-1].lower(), 
                                 index_col="#OTU ID")

In [9]:
class OTUDataKeeper:
    def __init__(self, name: str, df=pd.DataFrame()):
        self.name = name
        self.otu_df = df.fillna(0)
        self.alpha_diversity_metric_df = pd.DataFrame()

    def count_alpha_diversity_metrics(self):
        from skbio.diversity import alpha as a
        _FUNCTIONS = {"Distinct OTUs": a.observed_otus, "Shannon Entropy": a.shannon, 
                      "Berger-Parker Dominance": a.berger_parker_d, "Chao1 Richness": a.chao1, 
                      "Simpson Index": a.simpson}
        assert self.otu_df.shape[1] > 0
        # Without tree fit
        alpha_df = pd.concat([self.otu_df.apply(_FUNCTIONS[k], axis=1).rename(k) 
                              for k in _FUNCTIONS], axis=1, sort=False)
        alpha_df["Inverse Simpson Index"] = 1.0 / alpha_df["Simpson Index"]
        alpha_df["Gini–Simpson Index"] = 1.0 - alpha_df["Simpson Index"]
        # With tree fit
        otu_tree_df = fit_df_to_tree(self.otu_df)
        otu_tree_row_series = [otu_tree_df.loc[i, :] for i in otu_tree_df.index.values]
        faith_pd_series = Utilities.multi_core_queue(mp_count_faith_pd, otu_tree_row_series)
        otu_faith_pd_df = pd.DataFrame(faith_pd_series).set_index("patient_id")
        self.alpha_diversity_metric_df = pd.concat(
            [alpha_df, otu_faith_pd_df], axis=1, sort=False).rename_axis(
            index="patient_id", columns="alpha_diversity_metrics")

    def __repr__(self):
        return "OTUDataKeeper with name '{}', {} rows and {} columns".format(
            self.name, *self.otu_df.shape)

In [10]:
keepers = []
for raw_otu_df, raw_otu_name in zip([raw_stool_otu_df, raw_blood_otu_df], ["blood", "stool"]):
    otu_df = raw_otu_df.rename(columns={'"465x"': "465", '"715x"': "715"}).transpose().rename_axis(
            index="sample_name", columns="OTU").reset_index()
    otu_df["patient_id"] = otu_df["sample_name"].str.lower().replace('["b]+', "", regex=True)
    otu_df = otu_df.set_index("patient_id").drop("sample_name", axis=1)
    common_with_clinical_data_otu_df = otu_df.loc[np.intersect1d(*[i.index.values for i in (
        whole_raw_df, otu_df)])]
    keeper = OTUDataKeeper(name=raw_otu_name, df=common_with_clinical_data_otu_df)
    keeper.count_alpha_diversity_metrics()
    keepers.append(keeper)

In [11]:
common_with_both_samples_patient_ids = np.intersect1d(*[j.index.values for j in 
                                                        [i.otu_df for i in keepers]])
common_with_both_samples_otu_df = pd.DataFrame()
for keeper in keepers:
    otu_df = keeper.otu_df.loc[common_with_both_samples_patient_ids, :].reset_index()
    otu_df["sample_source"] = keeper.name
    common_with_both_samples_otu_df = pd.concat([common_with_both_samples_otu_df, otu_df], axis=0, 
                                                join="outer", ignore_index=True, sort=False)

common_with_both_samples_otu_df = common_with_both_samples_otu_df.set_index(
    ["patient_id", "sample_source"]).rename_axis(columns="OTU")
assert len([i for i in common_with_both_samples_otu_df if not i.lower().startswith("k__")]) == 1

In [12]:
def mp_count_beta_diversity(patient_id):
    beta_diversity_metrics = dict(patient_id=patient_id)
    # Without tree fit
    common_with_both_samples_otu_sub_df = common_with_both_samples_otu_df.loc[patient_id].fillna(0)
    otu_values_2d_array = common_with_both_samples_otu_sub_df.values
    beta_diversity_metrics["Manhattan Distance"] = distance.cityblock(*otu_values_2d_array)
    beta_diversity_metrics["Bray-Curtis Distance"] = distance.braycurtis(*otu_values_2d_array)
    beta_diversity_metrics["Jaccard Distance"] = distance.jaccard(*otu_values_2d_array)
    # With tree fit
    common_with_both_samples_otu_tree_sub_df = fit_df_to_tree(common_with_both_samples_otu_sub_df)
    otu_values_tree_2d_array = common_with_both_samples_otu_tree_sub_df.values
    beta_diversity_metrics["Unweighted UniFrac"] = skbio.diversity.beta.unweighted_unifrac(
        *otu_values_tree_2d_array, common_with_both_samples_otu_tree_sub_df.columns, otu_97_tree)
    beta_diversity_metrics["Weighted UniFrac"] = skbio.diversity.beta.weighted_unifrac(
        *otu_values_tree_2d_array, common_with_both_samples_otu_tree_sub_df.columns, otu_97_tree)
    return pd.Series(beta_diversity_metrics)

In [13]:
beta_diversity_series = Utilities.multi_core_queue(mp_count_beta_diversity, 
                                                   common_with_both_samples_patient_ids)
beta_diversity_df = pd.DataFrame(beta_diversity_series).set_index("patient_id").rename_axis(
    columns="beta_diversity_metrics")

In [14]:
keeper = [i for i in keepers if i.name == "stool"][0]
total_diversity_df = pd.concat([beta_diversity_df, keeper.alpha_diversity_metric_df], axis=1, 
                               join="inner", sort=False)

In [15]:
# Boilerplate1
# Build diversity - clinical - ELISA data correlation
data_prefix = "clinical, ELISA and diversity data"
for group_name in group_names:
    group_df = whole_raw_df.query("group == '{}'".format(group_name))
    merged_df = pd.concat([group_df.set_index("patient_id"), total_diversity_df], 
                          axis=1, join="inner", sort=False).rename_axis(
        columns=safe_capitalize(data_prefix))
    build_clustermap(data=merged_df, out_dir=correlation_dir, 
                     title=CORRELATION_TITLE_TEMPLATE.format(data_prefix, group_name))
    build_pairplot(data=merged_df, out_dir=correlation_dir, 
                   title=PAIRPLOT_TITLE_TEMPLATE.format(data_prefix, group_name))

for obesity_subgroup_name in obesity_subgroup_names:
    obesity_subgroup_df = whole_raw_df.query("subgroup == '{}'".format(obesity_subgroup_name))
    merged_df = pd.concat([obesity_subgroup_df.set_index("patient_id"), total_diversity_df], 
                      axis=1, join="inner", sort=False).rename_axis(
        columns=safe_capitalize(data_prefix))
    build_clustermap(data=merged_df, out_dir=correlation_dir, 
                     title=CORRELATION_TITLE_TEMPLATE.format(data_prefix, obesity_subgroup_name))
    build_pairplot(data=merged_df, out_dir=correlation_dir, 
                   title=PAIRPLOT_TITLE_TEMPLATE.format(data_prefix, obesity_subgroup_name))